In [6]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2
scikit-learn-intelex==20230228.214242


In [7]:
import pickle
import pandas as pd
import uuid

In [8]:
year =2022
month = 2
input_file = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet'
output_file = 'output/yellow_ride_id_prediction.parquet'
with open('/home/ubuntu/mlops-zoomcamp/04-deployment/homework4/models/lin_reg.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [9]:


def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    dicts = df[categorical].to_dict(orient='records')
    
    
    return df, dicts

In [10]:
def make_predictions():  
    df, dicts = read_data(input_file)  
    X_val = dv.transform(dicts)
    y_pred = model.predict(X_val)
    return y_pred

In [11]:
def new_dataframe(year, month):
    df, dicts = read_data(input_file)
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['prediction'] = make_predictions()
    print(df_result.prediction.std())
    df_result.to_parquet(output_file, engine='pyarrow', compression=None, index=False)
    return output_file

In [12]:
new_dataframe(2022, 2)

5.281400707361821


'output/yellow_ride_id_prediction.parquet'

In [13]:
# import os
# filename = 'lin_reg.bin'

# file_path = os.path.abspath(filename)
# print(file_path)